In [2]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import pandas as pd
import ctypes
import xml.etree.ElementTree as ET, urllib.request, gzip, io
from query_catalog_CBPs import find_CBPs

mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 20

%matplotlib inline

# Better looking figures
%config InlineBackend.figure_format = 'retina'

# Make cells narrower for better typography
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:65% !important; }</style>"))

In [3]:
# Load shared C library into Python via ctypes
lib = np.ctypeslib.load_library('../binary_c/python_API/libevolve', '.')

# Specify input and output types for called functions
lib.evolve_binary.argtypes = [ctypes.c_char_p]
lib.evolve_binary.restype = ctypes.c_void_p
lib.free_buffer.argtypes = [ctypes.c_void_p]
lib.free_buffer.restype = None

def evolve_binary(tmax, m1, m2, Z, P, e):
    """
    Evolves a binary system with the binary_c code.
    
    This function calls C code which uses the binary_c
    API to evolve a binary system. The output structure
    has to be defined in the C code.
    
    Parameters
    ----------
    tmax : float
        Maximum evolution time.
    m1 : float
        Mass of star 1.
    m2 : float
        Mass of star 2.
    Z : float
        Metallicity of the binary (assumed equal for both stars)
    P : float
        Inital orbital period of the binary.
    e : float
        Initial eccentricity.
        
    Returns
    -------
    ndarray
        Numpy array with the following output:
        t[Myr], m1[msun], m2[msun], R1[rsun], R2[rsun], e,
        P[years], a[rsun], type1, type2, snova, coel, merged, com
        
        type1 and type2 refer to stellar types. Possible stellar
        types are "Low mass MS", "MS", "Hertzsprung gap", "Giant branch",
        "Core Helium burning", "Early AGB", "TP AGB", "Naked Helium MS",
        "Naked Helium HG", "Naked Helium GB", "Helium WD", "CO WD", "ON WD",
        "NS","BH","Massless remnant". With Low mass MS designated by 0,
        MS by 1 and so on.
        
    """
    tmax=str(tmax); m1=str(m1); m2=str(m2); Z=str(Z); P=str(P); e=str(e)
    
    # Enter binary parameters
    params = bytes("binary_c M_1 " + m1 + " M_2 " + m2 + " metallicity " + Z\
              + " orbital_period " + P + " max_evolution_time " + tmax\
                   + " eccentricity " + e, 'utf8')
    # Call C function for system evolution
    ptr_to_output = lib.evolve_binary(params)
    
    # Read byte values of the output into memory
    values = ctypes.cast(ptr_to_output, ctypes.c_char_p).value
    
    # Free memory allocated by C code
    lib.free_buffer(ptr_to_output)

    string = str(values, 'utf-8')
    
    # Remove last part of string about element yields
    string = string[:string.rfind('XYIELD')]
    array = np.fromstring(string, sep=",")
    n_columns = 14
    array = array.reshape(int(len(array)/n_columns), n_columns)
    array[:, 6] *= 365.25 # convert P units to days
    array[:, 7] *= 0.00465 # convert a units to au

    # Create DataFrame object
    columns = ['t', 'M1', 'M2', 'R1', 'R2', 'e',
        'P', 'a', 'type1', 'type2', 'snova', 'coel','merged','com']
    
    df = pd.DataFrame(array, columns=columns, index=array[:, 0])

    return df

def outcomes(df):
    """
    Iterates through the stellar evolution code output
    and prints important events and final parametars of the system.
    """
    
    final_conf = ["Deeply or fully convective low mass MS","Main Sequence","Hertzsprung Gap",
              "First Giant Branch","Core Helium Burning","First Asymptotic Giant Branch",
              "Second Asymptotic Giant Branch","Main Sequence Naked Helium",
              "Hertzsprung Gap Naked Helium","Giant Branch Naked Helium",
              "Helium White Dwarf","Carbon/Oxygen White Dwarf","Oxygen/Neon White Dwarf",
              "Neutron","Black Hole","Massless"]
    
    print("FINAL CONFIGURATION:\n",final_conf[int(df['type1'].iloc[-1])],"+",
          final_conf[int(df['type2'].iloc[-1])])
    
    events = ['CE occured', 'Merger occured', 'Coelesence occured', 'SN occured', 'None']
    if np.any(df['com']) == True:
        print(events[0])
    if np.any(df['merged']) == True:
        print(events[1])
    if np.any(df['coel']) == True:
        print(events[2])
    if np.any(df['snova']) == True:
        print(events[3])
        
    print("t_final = ", df['t'].iloc[-1], "Myr")
    print("M1 = ", df['M1'].iloc[-1]," M2 =", df['M2'].iloc[-1])
    print("P = ", df['M2'].iloc[-1], "days","\n")

In [10]:
test = evolve_binary(13700, 1.4
                           , 0.8, 0.02,
                           800,
                           0.3)
outcomes(test)
test

FINAL CONFIGURATION:
 Carbon/Oxygen White Dwarf + Main Sequence
t_final =  13700.0 Myr
M1 =  0.514821  M2 = 0.80831
P =  0.80831 days 



,t,M1,M2,R1,R2,e,P,a,type1,type2,snova,coel,merged,com
0.000000,0.000000,1.400000,0.800000,1.372330,0.729209,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
1.311578,1.311578,1.400000,0.800000,1.372490,0.729214,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
2.623157,2.623157,1.400000,0.800000,1.372660,0.729219,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
3.934735,3.934735,1.400000,0.800000,1.372820,0.729225,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
5.246313,5.246313,1.400000,0.800000,1.372990,0.729230,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
6.557892,6.557892,1.400000,0.800000,1.373150,0.729235,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
7.869470,7.869470,1.400000,0.800000,1.373320,0.729240,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
9.181049,9.181049,1.400000,0.800000,1.373480,0.729246,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
10.492627,10.492627,1.400000,0.800000,1.373650,0.729251,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0
11.804205,11.804205,1.400000,0.800000,1.373820,0.729256,0.3,800.021685,2.192498,1.0,1.0,0.0,0.0,0.0,0.0


# Evolution of Kepler CBP hosting stars

In [4]:
cbp_transit = find_CBPs('transit')

<class 'xml.etree.ElementTree.ElementTree'>
Total number of CBPs:  29 

2M 1938+4603 b - timing
DP Leo b - timing
FL Lyr b - timing
FW Tau b - imaging
HU Aqr (AB) b - timing
HU Aqr (AB) c - timing
HU Aqr (AB) d - timing
HW Vir (AB) b - timing
Kepler-1647 b - transit
Kepler-16 (AB) b - transit
Kepler-34 (AB) b - transit
Kepler-35 (AB) b - transit
Kepler-38 (AB) b - transit
Kepler-413 b - transit
Kepler-47 (AB) b - transit
Kepler-47 (AB) c - transit
KIC 9632895 b - transit
KOI-2939 b - transit
NN Ser (AB) c - timing
NN Ser (AB) d - timing
NSVS 14256825 c - timing
NSVS 14256825 d - timing
NY Virginis (AB) b - timing
PH-1 A(ab) b - transit
PSR B1620-26 b - timing
Ross 458 C - imaging
ROXs 42 B b - imaging
RR Cae (AB) b - timing
SR 12 C - imaging

Systems with multiple CBPs: 

HU Aqr , 3 planets
Kepler-47 , 2 planets
NN Ser A , 2 planets
NSVS 14256825 A , 2 planets


In [5]:
def evolve_kepler_stars():
    
    stars = ['Kepler-34 (AB) b','Kepler-35 (AB) b', 'Kepler-413 b','Kepler-47 (AB) b',
            'KIC 9632895 b','KOI-2939 b','PH-1 A(ab) b']

    for i, s in enumerate(stars):
        output = evolve_binary(13700, cbp_transit.loc[stars[i],'mstar1']
                           , cbp_transit.loc[stars[i],'mstar2'], 0.02,
                           cbp_transit.loc[stars[i],'P_binary'],
                           cbp_transit.loc[stars[i],'e_binary'])
        print(s, ":")
        outcomes(output)
        # Ignore post CE evolution (e < 0)
#         mask = output['e'] >= 0
        
#         fig, ax = plt.subplots(4,1, figsize=(10,20))
#         # fig.subplots_adjust(hspace=.35)
#         for a in ax.ravel():
#             a.grid(True)
#         #     a.set_xlim(3600,3880)

#         ax[0].plot(output['t'].loc[mask], output['M1'].loc[mask])
#         ax[0].plot(output['t'].loc[mask], output['M2'].loc[mask])
#         ax[0].set_ylabel(r'mass [msun]')
#         ax[1].plot(output['t'].loc[mask], output['R1'].loc[mask])
#         ax[1].plot(output['t'].loc[mask], output['R2'].loc[mask])
#         ax[1].set_ylabel(r'radius [rsun]')
#         ax[2].plot(output['t'].loc[mask], output['P'].loc[mask])
#         ax[2].set_ylabel(r'period [day]')
#         ax[3].plot(output['t'].loc[mask], output['e'].loc[mask])
#         ax[3].set_ylabel(r'eccentricity')
#         ax[3].set_xlabel('time [Myr]')

#         plt.savefig(s+'.pdf')

evolve_kepler_stars()

Kepler-34 (AB) b :
FINAL CONFIGURATION:
 Massless + Carbon/Oxygen White Dwarf
Merger occured
t_final =  13700.0 Myr
M1 =  0.0  M2 = 0.372177
P =  0.372177 days 

Kepler-35 (AB) b :
FINAL CONFIGURATION:
 Main Sequence + Main Sequence
t_final =  13700.0 Myr
M1 =  0.81  M2 = 0.89
P =  0.89 days 

Kepler-413 b :
FINAL CONFIGURATION:
 Deeply or fully convective low mass MS + Main Sequence
t_final =  13700.0 Myr
M1 =  0.5423  M2 = 0.82
P =  0.82 days 

Kepler-47 (AB) b :
FINAL CONFIGURATION:
 Massless + Carbon/Oxygen White Dwarf
Merger occured
t_final =  13700.0 Myr
M1 =  0.0  M2 = 0.581407
P =  0.581407 days 

KIC 9632895 b :
FINAL CONFIGURATION:
 Deeply or fully convective low mass MS + Main Sequence
t_final =  13700.0 Myr
M1 =  0.1938  M2 = 0.934
P =  0.934 days 

KOI-2939 b :
FINAL CONFIGURATION:
 Massless + Carbon/Oxygen White Dwarf
Merger occured
t_final =  13700.0 Myr
M1 =  0.0  M2 = 0.573052
P =  0.573052 days 

PH-1 A(ab) b :
FINAL CONFIGURATION:
 Massless + Carbon/Oxygen White Dwar

# Evolution in the period-eccentricity space

In [1]:
# Plot only post MS evolution
# lowMS = output[:, -2] == 0
# MS = output[:, -2] == 1
# HG = output[:, -2] == 2 
# GB = output[:, -2] == 3
# CHeb = output[:, -2] == 4 
# EAGB = output[:, -2] == 5
# TPAGB = output[:, -2] == 6
# mask = lowMS + MS + HG + GB + CHeb

# def plot_period_eccentricity():
P_planet = 5.6*output[0, 6]

fig, ax = plt.subplots(figsize=(10,7))
ax.plot(output[:, 6], output[:, 5], 'k.')
ax.axvline(x=P_planet/6., color='red', label='6:1')
ax.axvline(x=P_planet/7., color='C0', label='7:1')
ax.axvline(x=P_planet/8., color='C1', label='8:1')
ax.axvline(x=P_planet/9., color='C2', label='9:1')
ax.grid(True)
ax.set_xlabel(r'binary period [days]')
ax.set_ylabel(r'binary eccentricity')
ax.legend()
ax.set_xlim(P_planet/10, output[0, 6])
ax.set_ylim(0, .5)

NameError: name 'output' is not defined

In [ ]:
# def plot_resonances(ax, m1, m2, m3, e_i, e, P_i, P_o):
#     def resonance_width(m1, m2, m3, n, e_i, e_o):
#         m123 = m1 + m2 + m3
#         m12 = m1 + m2
#         xi = np.arccosh(1/e_o) - np.sqrt(1 - e_o**2)
#         tmp = (6*0.71**.5/((2*np.pi)**(1/4.)))*\
#                 (m3/m123 + n**(2/3.) *(m12/m123)**(2/3.)*(m1*m2/m12**2))**.5 *\
#                 (e_i**.5/e_o)*(1 - 13/24.*e_i**2.)**.5*(1 - e_o**2)**(3/8.)*\
#                 n**(3/4.)*np.exp(-n*xi/2.);
#         return tmp
    
#     e_o = np.linspace(0, 1, 1000) 
    
#     for n in range(2, 9):
#         width = resonance_width(m1, m2, m3, n, e_i, e_o)
#         ax.fill_betweenx(e_o, -width + n, width + n, facecolor='C0', alpha=0.3)
        
#     ax.grid(True)
# #     ax.set_xlim(0, 12)
#     ax.set_xlabel(r'$\nu_i/\nu_o$')
#     ax.set_ylabel(r'$e_o$')
#     ax.set_xticks(np.arange(0, 13))   
#     ax.scatter(P_o/P_i, e, color='black')
#     ax.legend()

# fig, ax = plt.subplots(figsize=(7,5))
# x = 1985
# plot_resonances(ax, output[x, 1], output[x, 2], 1e-04,
#                 output[x, 5], 0.04, output[x, 6]*365.2, 131.4)